In [1]:
import numpy as np
import pandas as pd
import os
import math
from datetime import date as pdate
import statistics

In [2]:
file_path = os.path.abspath("data/hist_data.xlsm")
notional_stock = 1000000
notional_swap = 100000000
swap_rate = 0.042
day_count_frac = 1

In [3]:
def get_swap_data(file_path):
    """
    read SOFR curve data from excel file 

    Args:
    file_path(str): path to the excel file

    Returns:
    pd.DataFrame: preprocessed SOFR curve data
    """
    df_sofr_curve_raw = pd.read_excel(file_path, sheet_name="SofrCurve", engine = 'openpyxl', parse_dates=False)
    # clean up column names (timestamp)
    df_sofr_curve_raw.columns = [str(col) for col in df_sofr_curve_raw.columns]
    df_sofr_curve_raw.columns = [col.split()[0] if '-' in col else col for col in df_sofr_curve_raw.columns]
    df_sofr_curve_raw.drop(columns=['Tenor'], inplace=True)
    df_sofr_curve_raw['T'] = df_sofr_curve_raw['T'].astype(int) # convert T to int
    # extract out relevant zero curves (1Y-10Y)
    df_sofr_curve = df_sofr_curve_raw.iloc[6:16].copy()
    
    return df_sofr_curve


In [4]:
def get_stocks_data(file_path):
    """
    read stock data from excel file

    Args:
    file_path(str): path to the excel file

    Returns: 
    pd.DataFrame: preprocessed stock data
    """
    sheet_names = ["AAPL", "MSFT", "F", "BAC"]
    df_stocks = {}
    # read stock data in sheet names lst 
    for sheet_name in sheet_names:
        df_stocks[sheet_name] = pd.read_excel(file_path, sheet_name=sheet_name, engine = 'openpyxl', parse_dates=False)
        # rename columns in sheet to avoid dupes 
        df_stocks[sheet_name].rename(columns = {"Adj Close": f"Adj Close_{sheet_name}"}, inplace = True)
    # merge stock data into dataframe based on date
    df_stocks_merged = pd.merge(df_stocks["AAPL"], df_stocks["MSFT"], on="Date", how="inner")
    df_stocks_merged = pd.merge(df_stocks_merged, df_stocks["F"], on="Date", how="inner")
    df_stocks_merged = pd.merge(df_stocks_merged, df_stocks["BAC"], on="Date", how="inner")

    df_stocks_merged['Date'] = pd.to_datetime(df_stocks_merged['Date'])

    # calculate relative return for each stock
    for col in df_stocks_merged.columns:
        if "Adj Close" in col:  # Apply only to price columns
            df_stocks_merged[f"Return_{col.split('_')[-1]}"] = df_stocks_merged[col].pct_change()
    
    df_stocks_merged = df_stocks_merged.iloc[:,5:] # filter returns only
    df_stocks_merged = df_stocks_merged.iloc[1:] # remove the first row since it is NaN

    return df_stocks_merged

In [5]:
df_stocks_merged = get_stocks_data(file_path)
df_stocks_merged

,Return_AAPL,Return_MSFT,Return_F,Return_BAC
1,-0.017543,-0.017059,0.002244,0.004439
2,-0.037305,-0.035368,-0.025373,-0.003039
3,-0.042405,-0.026579,0.015314,-0.005542
4,-0.001947,0.033326,0.018854,0.025077
5,0.003902,0.029270,0.014064,0.005980
...,...,...,...,...
246,0.002543,0.003674,-0.007840,-0.003911
247,-0.013492,0.030678,0.013169,0.003141
248,-0.024606,-0.037514,-0.016464,0.022309
249,0.007969,0.005856,-0.122467,-0.036371


In [6]:
df_sofr_curve = get_swap_data(file_path)
df_sofr_curve

,T,2022-10-31,2022-11-01,2022-11-02,2022-11-03,2022-11-04,2022-11-07,2022-11-08,2022-11-09,2022-11-10,...,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25,2023-10-26,2023-10-27,2023-10-30
6,1,0.046449,0.046970,0.047203,0.047894,0.047500,0.047929,0.047589,0.047479,0.045989,...,0.053225,0.053163,0.052750,0.052348,0.052389,0.052503,0.052653,0.052243,0.052115,0.052245
7,2,0.044583,0.045022,0.045496,0.046594,0.046097,0.046633,0.046046,0.045585,0.042912,...,0.049497,0.049611,0.049204,0.048366,0.048027,0.048399,0.048791,0.048044,0.047758,0.047904
8,3,0.042002,0.042344,0.042749,0.043833,0.043385,0.043884,0.043209,0.042645,0.039631,...,0.046978,0.047304,0.047169,0.046151,0.045582,0.045999,0.046595,0.045645,0.045284,0.045429
9,4,0.040318,0.040614,0.040868,0.041825,0.041503,0.041916,0.041182,0.040704,0.037584,...,0.045605,0.046034,0.046195,0.045156,0.044508,0.044850,0.045594,0.044538,0.044200,0.044345
10,5,0.039297,0.039522,0.039675,0.040507,0.040303,0.040635,0.039857,0.039489,0.036386,...,0.044929,0.045414,0.045751,0.044758,0.044048,0.044305,0.045200,0.044086,0.043762,0.043928
11,6,0.038534,0.038658,0.038776,0.039537,0.039421,0.039734,0.038881,0.038647,0.035572,...,0.044567,0.045049,0.045511,0.044604,0.043871,0.044036,0.045017,0.043893,0.043636,0.043794
12,7,0.037979,0.038007,0.038117,0.038820,0.038782,0.039111,0.038192,0.038085,0.035027,...,0.044333,0.044795,0.045361,0.044548,0.043806,0.043889,0.044911,0.043808,0.043643,0.043779
13,8,0.037648,0.037605,0.037704,0.038328,0.038373,0.038715,0.037781,0.037765,0.034725,...,0.044178,0.044635,0.045287,0.044554,0.043784,0.043816,0.044867,0.043795,0.043702,0.043828
14,9,0.037465,0.037368,0.037458,0.038002,0.038128,0.038473,0.037560,0.037612,0.034598,...,0.044081,0.044545,0.045266,0.044606,0.043798,0.043798,0.044876,0.043829,0.043793,0.043915
15,10,0.037353,0.037209,0.037300,0.037796,0.037988,0.038327,0.037447,0.037562,0.034588,...,0.044033,0.044508,0.045282,0.044690,0.043853,0.043826,0.044926,0.043898,0.043908,0.044023


In [7]:
def get_discount_curve(zero_rates, tenors):
    """
    discount factors based on zero rates

    Args:
    zero_rates(list): list of zero rates
    tenors(list): list of tenors

    Returns:
    np.array: discount factors
    """
    assert len(zero_rates) == len(tenors), f"Expect {len(tenors)} zero rates, got {len(zero_rates)}."
    Z = np.array(zero_rates)
    T = np.array(tenors)
    return np.exp(-Z*T)


def get_forward_curve(zero_rates, tenors, day_count_frac=1):
    """
    forward rates based on zero rates

    Args:
    zero_rates(list): list of zero rates
    tenors(list): list of tenors
    day_count_frac(float): day count fraction   

    Returns:
    np.array: forward rates
    """
    DF = get_discount_curve(zero_rates, tenors)
    DF_start = np.concatenate([[1], DF[:-1]]) # first DF=1. we dont consider forward swap here
    DF_end = DF
    F = (DF_start - DF_end) / (DF_end * day_count_frac)
    return F
    

def get_payer_swap_pv(zero_rates, tenors, forward_rates=None, swap_rate=0.042, day_count_frac=1, notional=100000000):
    """
    calculate the present value of a payer swap

    Args:
    zero_rates(list): list of zero rates
    tenors(list): list of tenors
    forward_rates(list): list of forward rates
    swap_rate(float): fixed swap rate
    day_count_frac(float): day count fraction
    notional(float): notional amount

    Returns:
    float: present value of a swap
    """
    DF = get_discount_curve(zero_rates, tenors)
    F = get_forward_curve(zero_rates, tenors) if forward_rates is None else np.array(forward_rates)
    pv_fix = swap_rate*sum(day_count_frac*DF)
    pv_flt = sum(day_count_frac*F*DF)
    return notional*(pv_flt - pv_fix)


def swap_pnl_1d_full(zero_rates_t0, zero_rates_t1, tenors):
    """
    1d PNL of swap - full revaluation

    Args:
    zero_rates_t0(list): list of zero rates at t0 (current zero rates)
    zero_rates_t1(list): list of zero rates at t1 (previous historical zero rates)
    tenors(list): list of tenors

    Returns:
    float: 1d PNL of swap
    """

    pv_t0 = get_payer_swap_pv(zero_rates_t0, tenors)
    pv_t1 = get_payer_swap_pv(zero_rates_t1, tenors)
    return pv_t1 - pv_t0


def swap_pnl_1d_sens(zero_rates_t0, zero_rates_chng, tenors, swap_rate = 0.042, notional=100*1000000):
    """
    1d PNL of swap - sensitivity based approach

    Args:
    zero_rates_t0(list): list of zero rates at t0 
    zero_rates_chng(list): list of zero rates change
    tenors(list): list of tenors
    swap_rate(float): fixed swap rate
    notional(float): notional amount

    Returns:
    float: 1d PNL of swap
    """ 
    zero_rates_chng = np.array(zero_rates_chng)
    tenors = np.array(tenors)
    DF_t0 = get_discount_curve(zero_rates_t0, tenors) # exponential discount factor
    W = (notional * swap_rate * tenors * DF_t0)# weight of risk factors in PnL sensitivity
    W[-1] = notional * (1+swap_rate) * tenors[-1] * DF_t0[-1]
    return W @ zero_rates_chng

In [8]:
### sanity check - break down of function -> pv swap value
# def calc_pv_swap(df_sofr_curve, notional_swap, swap_rate, day_count_frac):
#     """
#     calculate the present value of a swap

#     Args:
#     df_sofr_curve(pd.DataFrame): SOFR curve data
#     notional_swap(float): notional amount of the swap
#     swap_rate(float): fixed rate of the swap
#     day_count_frac(float): day count fraction (annual)

#     Returns:
#     float: present value of the swap
#     """
#     # filter relevant columns (T and Zero rate)
#     df_sofr_curve_filter = df_sofr_curve.iloc[:, [0, -1]]
#     last_col =df_sofr_curve_filter.columns[-1]
#     df_sofr_curve_filter.rename(columns={last_col: 'ZeroRate'}, inplace=True)

#     df_sofr_curve_fixed = df_sofr_curve_filter.copy()
#     df_sofr_curve_float = df_sofr_curve_filter.copy()

#     ## calculate fixed leg PV
#     # discount factor 
#     df_sofr_curve_fixed['DF'] = np.exp(-df_sofr_curve_fixed['ZeroRate'] * df_sofr_curve_fixed['T'])
#     # cash flows
#     df_sofr_curve_fixed['CashFlow'] = notional_swap * swap_rate * day_count_frac
#     swap_fixed_leg = np.sum(df_sofr_curve_fixed['CashFlow'] * df_sofr_curve_fixed['DF'])
#     print("Swap Fixed Leg: ", swap_fixed_leg)

#     ## calculate floating leg PV
#     df_sofr_curve_float = df_sofr_curve_float.reset_index(drop=True)
#     df_sofr_curve_float['DF_end'] = np.exp(-df_sofr_curve_float['ZeroRate']*df_sofr_curve_float['T'])
#     # assign the end discount factor of the last tenor to the start discount factor of the next tenor
#     df_sofr_curve_float['DF_start'] = df_sofr_curve_float['DF_end'].shift(1)
#     # assign the start discount factor of the first tenor to 1
#     df_sofr_curve_float.loc[0, 'DF_start'] = 1
#     df_sofr_curve_float['F'] = (df_sofr_curve_float['DF_start']/df_sofr_curve_float['DF_end'] - 1)
#     df_sofr_curve_float['CashFlow'] = notional_swap * df_sofr_curve_float['F'] * day_count_frac
#     swap_float_leg = np.sum(df_sofr_curve_float['CashFlow'] * df_sofr_curve_float['DF_end'])
#     print("Swap Float Leg: ", swap_float_leg)
    
#     pv_swap = float(swap_float_leg - swap_fixed_leg)
#     print("Present Value of Swap: ", pv_swap)
    
#     return pv_swap

In [9]:
df_sofr_curve.set_index(df_sofr_curve.iloc[:,0].values)
df_sofr_curve

,T,2022-10-31,2022-11-01,2022-11-02,2022-11-03,2022-11-04,2022-11-07,2022-11-08,2022-11-09,2022-11-10,...,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25,2023-10-26,2023-10-27,2023-10-30
6,1,0.046449,0.046970,0.047203,0.047894,0.047500,0.047929,0.047589,0.047479,0.045989,...,0.053225,0.053163,0.052750,0.052348,0.052389,0.052503,0.052653,0.052243,0.052115,0.052245
7,2,0.044583,0.045022,0.045496,0.046594,0.046097,0.046633,0.046046,0.045585,0.042912,...,0.049497,0.049611,0.049204,0.048366,0.048027,0.048399,0.048791,0.048044,0.047758,0.047904
8,3,0.042002,0.042344,0.042749,0.043833,0.043385,0.043884,0.043209,0.042645,0.039631,...,0.046978,0.047304,0.047169,0.046151,0.045582,0.045999,0.046595,0.045645,0.045284,0.045429
9,4,0.040318,0.040614,0.040868,0.041825,0.041503,0.041916,0.041182,0.040704,0.037584,...,0.045605,0.046034,0.046195,0.045156,0.044508,0.044850,0.045594,0.044538,0.044200,0.044345
10,5,0.039297,0.039522,0.039675,0.040507,0.040303,0.040635,0.039857,0.039489,0.036386,...,0.044929,0.045414,0.045751,0.044758,0.044048,0.044305,0.045200,0.044086,0.043762,0.043928
11,6,0.038534,0.038658,0.038776,0.039537,0.039421,0.039734,0.038881,0.038647,0.035572,...,0.044567,0.045049,0.045511,0.044604,0.043871,0.044036,0.045017,0.043893,0.043636,0.043794
12,7,0.037979,0.038007,0.038117,0.038820,0.038782,0.039111,0.038192,0.038085,0.035027,...,0.044333,0.044795,0.045361,0.044548,0.043806,0.043889,0.044911,0.043808,0.043643,0.043779
13,8,0.037648,0.037605,0.037704,0.038328,0.038373,0.038715,0.037781,0.037765,0.034725,...,0.044178,0.044635,0.045287,0.044554,0.043784,0.043816,0.044867,0.043795,0.043702,0.043828
14,9,0.037465,0.037368,0.037458,0.038002,0.038128,0.038473,0.037560,0.037612,0.034598,...,0.044081,0.044545,0.045266,0.044606,0.043798,0.043798,0.044876,0.043829,0.043793,0.043915
15,10,0.037353,0.037209,0.037300,0.037796,0.037988,0.038327,0.037447,0.037562,0.034588,...,0.044033,0.044508,0.045282,0.044690,0.043853,0.043826,0.044926,0.043898,0.043908,0.044023


In [10]:
## swap pnl full revaluation and sensitivity based approach
tenors = df_sofr_curve.iloc[:,0].values

# historical date cols 
hist_swaps = df_sofr_curve.columns[1:] # exclude first column (tenors)

swap_hist_full_pnl = []
swap_hist_sensi_pnl = []

# iterate over historical sofr swap rates
for i in range(1, len(hist_swaps)):
    zero_rates_t0 = df_sofr_curve.iloc[:, i].values # current swap zero rate 10th year tenor
    zero_rates_t1 = df_sofr_curve.iloc[:, i-1].values # past swap zero rates 1-9th year tenor
    zero_rates_delta = zero_rates_t0 - zero_rates_t1 # delta 
    swap_hist_full_pnl.append(swap_pnl_1d_full(zero_rates_t0, zero_rates_t1, tenors))
    swap_hist_sensi_pnl.append(swap_pnl_1d_sens(zero_rates_t0, zero_rates_delta, tenors))
# print(np.sum(swap_hist_full_pnl))
# print(np.sum(swap_hist_sensi_pnl))
# print(swap_hist_full_pnl[:10])
# print(swap_hist_sensi_pnl[:10])
# print(zero_rates_delta[:10])

In [11]:
ten_year_sofr_curve = df_sofr_curve.drop(columns=["T"]).T
ten_year_sofr_curve.columns = df_sofr_curve['T']
ten_year_sofr_curve

T,1,2,3,4,5,6,7,8,9,10
2022-10-31,0.046449,0.044583,0.042002,0.040318,0.039297,0.038534,0.037979,0.037648,0.037465,0.037353
2022-11-01,0.046970,0.045022,0.042344,0.040614,0.039522,0.038658,0.038007,0.037605,0.037368,0.037209
2022-11-02,0.047203,0.045496,0.042749,0.040868,0.039675,0.038776,0.038117,0.037704,0.037458,0.037300
2022-11-03,0.047894,0.046594,0.043833,0.041825,0.040507,0.039537,0.038820,0.038328,0.038002,0.037796
2022-11-04,0.047500,0.046097,0.043385,0.041503,0.040303,0.039421,0.038782,0.038373,0.038128,0.037988
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.052503,0.048399,0.045999,0.044850,0.044305,0.044036,0.043889,0.043816,0.043798,0.043826
2023-10-25,0.052653,0.048791,0.046595,0.045594,0.045200,0.045017,0.044911,0.044867,0.044876,0.044926
2023-10-26,0.052243,0.048044,0.045645,0.044538,0.044086,0.043893,0.043808,0.043795,0.043829,0.043898
2023-10-27,0.052115,0.047758,0.045284,0.044200,0.043762,0.043636,0.043643,0.043702,0.043793,0.043908


In [12]:
sofr_curve_t0 = ten_year_sofr_curve.iloc[-1]
sofr_curve_t0
pct_change_sofr_rates = ten_year_sofr_curve.diff().dropna().values

swap_hist_full_pnl = []
swap_hist_sensi_pnl = []

for current_sofr_curve in pct_change_sofr_rates:
    zero_rates_delta = np.array(current_sofr_curve)
    zero_rates_t1 = np.array(sofr_curve_t0) + zero_rates_delta
    pnl_full_evaulation = swap_pnl_1d_full(
        zero_rates_t0=sofr_curve_t0,
        zero_rates_t1=zero_rates_t1,
        tenors=tenors
    )
    pnl_swap_evaluation = swap_pnl_1d_sens(
        zero_rates_t0=sofr_curve_t0,
        zero_rates_chng=zero_rates_delta,
        tenors=tenors
    )
    swap_hist_full_pnl.append(pnl_full_evaulation)
    print(pnl_full_evaulation)
    swap_hist_sensi_pnl.append(pnl_swap_evaluation)
np.shape(swap_hist_full_pnl)

-79854.56466061389
85728.00273665693
440438.32198618073
111739.07747985562
279960.10813917685
-711893.6580142009
47993.531713097356
-2459267.732617787
182761.2158311177
-619010.5297204263
-951827.6164579452
668630.4582758709
435154.73433946026
117612.72064245725
-751451.8104459133
-364115.6111617193
-49693.23961608298
-46994.05137257837
294766.92075294256
-426548.5140266756
-1339805.246240816
-167024.61604356486
815479.6107719629
-852606.3256082239
-699411.0947710297
783371.9285207265
634360.2118005776
309218.1907489612
-925792.5558067537
32481.39165584743
-305495.06718171015
226683.3156882641
720249.2858196185
599348.9314237488
89424.99841707014
-22556.686046609655
590202.5483640106
744154.6406114274
291904.7496085237
-341393.2271747575
-193858.88575990638
-384772.5549195546
-586933.7629522963
121292.75653612334
-1209550.8256788512
-443400.4244774524
770880.735299767
-360325.41991028446
-762137.5445773569
509821.73152220296
60420.31626144471
-1049900.908576984
73855.99941457389
667502

(250,)

In [13]:
ten_year_sofr_curve.diff().dropna()

T,1,2,3,4,5,6,7,8,9,10
2022-11-01,0.000521,0.000439,0.000342,0.000296,0.000225,0.000124,0.000029,-0.000043,-9.724439e-05,-0.000144
2022-11-02,0.000234,0.000474,0.000404,0.000254,0.000153,0.000118,0.000109,0.000099,8.994989e-05,0.000091
2022-11-03,0.000690,0.001098,0.001085,0.000957,0.000832,0.000761,0.000704,0.000624,5.446595e-04,0.000495
2022-11-04,-0.000394,-0.000497,-0.000448,-0.000322,-0.000204,-0.000116,-0.000038,0.000045,1.258218e-04,0.000192
2022-11-07,0.000429,0.000536,0.000499,0.000413,0.000332,0.000313,0.000328,0.000342,3.452884e-04,0.000339
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000114,0.000371,0.000417,0.000343,0.000257,0.000165,0.000083,0.000032,2.757643e-07,-0.000027
2023-10-25,0.000150,0.000393,0.000596,0.000743,0.000894,0.000981,0.001022,0.001051,1.077486e-03,0.001100
2023-10-26,-0.000410,-0.000747,-0.000950,-0.001055,-0.001114,-0.001124,-0.001102,-0.001073,-1.046232e-03,-0.001028
2023-10-27,-0.000129,-0.000286,-0.000361,-0.000339,-0.000324,-0.000257,-0.000166,-0.000093,-3.667796e-05,0.000011


In [14]:
# stocks full revaluation 1d pnl evaluation 
def stocks_pnl1d_full(stocks_returns, w = [1e6, 1e6, 1e6, 1e6]):
    """
    stocks_returns: DataFrame of stocks returns
    w: list of notional weights
    """
    return (w[0]*((1+stocks_returns[0])-1) + w[1]*((1+stocks_returns[1])-1) + w[2]*((1+stocks_returns[2])-1) + w[3]*((1+stocks_returns[3])-1))

# stocks sensitivity based 1d pnl evaluation 
def stocks_pnl1d_sensi(stocks_returns, w = [1e6, 1e6, 1e6, 1e6]):
    """
    stocks_returns: DataFrame of stocks returns
    w: list of notional weights
    """
    return stocks_returns @ w

In [15]:
# historical VaR
confidence_level = 95

df_stocks_merged = get_stocks_data(file_path)
stocks_pnl1d_full_hist = [stocks_pnl1d_full(s) for s in df_stocks_merged.values]   
stocks_pnl1d_sensi_hist = [stocks_pnl1d_sensi(s) for s in df_stocks_merged.values]
pnl1d_full_hist = np.array(stocks_pnl1d_full_hist) + np.array(swap_hist_full_pnl)
var1d_full_hist = np.abs(np.percentile(pnl1d_full_hist, 100-confidence_level))

# TODO: check whether sensitivity based approach is correct - why is it larger than full reval?
pnl1d_sensi_hist = np.array(stocks_pnl1d_sensi_hist) + (np.array(swap_hist_sensi_pnl).reshape(-1,1)) 
var1d_sensi_hist = np.abs(np.percentile(pnl1d_sensi_hist, 100-confidence_level))

print("")
print("")
print("============================================================================================================================")
print("Historical VaR:")
print(f"VaR [1d, {confidence_level}%], Full Revaluation: {var1d_full_hist:,.0f}") 
print(f"VaR [1d, {confidence_level}%], Sensitivity: {var1d_sensi_hist:,.0f}") 
print("============================================================================================================================")




Historical VaR:
VaR [1d, 95%], Full Revaluation: 966,173
VaR [1d, 95%], Sensitivity: 972,734


In [16]:
print(pnl1d_full_hist)

[-1.07773367e+05 -1.53572844e+04  3.81226868e+05  1.87047565e+05
  3.33175336e+05 -6.99168092e+05 -5.00285847e+04 -2.17857532e+06
  2.48937388e+05 -6.88127895e+05 -9.22738765e+05  6.25132146e+05
  4.43553037e+05  1.25780658e+05 -7.69323742e+05 -3.23716348e+05
 -3.30020797e+04 -6.09943588e+04  1.98211031e+05 -4.48356769e+05
 -1.19570593e+06 -1.82647906e+05  7.84920379e+05 -9.58745270e+05
 -7.95204593e+05  7.45094249e+05  6.51255902e+05  3.00503714e+05
 -8.48460122e+05  6.55123167e+04 -3.40846681e+05  9.87286779e+04
  6.16279683e+05  5.41988997e+05  7.97464481e+04  5.43784440e+04
  4.92129176e+05  7.50514127e+05  2.58375554e+05 -3.97276856e+05
 -7.27328213e+04 -3.80045528e+05 -6.09264577e+05  1.34917397e+05
 -1.23185987e+06 -3.57903121e+05  7.78337727e+05 -3.29644790e+05
 -6.73571618e+05  5.39346819e+05  4.27394475e+04 -1.06054693e+06
  5.90096848e+03  6.18427417e+05  3.32990191e+05 -3.72754127e+05
  4.05206669e+04  2.72721669e+05  3.24660566e+05  2.96664756e+05
 -2.78293310e+05 -8.87178